In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data,datasets

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
DEVICE 

device(type='cuda')

In [0]:
## 하이퍼파라미터 설정
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 50

In [0]:
## IMDB 데이터셋 로딩 후 텐서로 변환
TEXT = data.Field(sequential=True,batch_first=True,lower=True)
LABEL = data.Field(sequential=False,batch_first=True)
## sequential => 데이터가 순차적인 데이터셋인지 
## batch_first => 파라미터로 신경망에 입력되는 텐서의 첫번째 차원값이 batch_size가 되도록 
## lower => 텍스트 데이터 속 모든 영문 알파벳이 소문자가 되도록 

trainset, testset = datasets.IMDB.splits(TEXT,LABEL)


In [0]:

## word embadding에 필요한 단어 사전을 만듬

TEXT.build_vocab(trainset,min_freq=5)
LABEL.build_vocab(trainset)


In [0]:
## 검증 SET이 없어서 TRAINSET에서 VALID SET 나눠서 사용한다잉 ~ 
trainset,valset = trainset.split(split_ratio=0.8)

## batch 단위로 쪼개서 학습을 진행 한다 
## iter를 enumerate()함수에 입력시켜 루프를 구현하면 루프 때마다 전체 데이터셋에서 배치단위의 데이터가 생성 
train_iter,val_iter,test_iter = data.BucketIterator.splits(
    (trainset, valset,testset),
    batch_size =BATCH_SIZE,
    shuffle=True, repeat=False 
)


In [42]:
## 사전 속 단어들으 ㅣ개수와 레이블으 ㅣ수를 정해주는 변수 
vocab_size = len(TEXT.vocab)
n_classes = 2 ## 1 or 2 임으로 

print("[학습셋] : {} [valset] : {} [testset] : {} [단어수] : {} [class ]: {}".format(len(trainset),len(valset),len(testset),vocab_size,n_classes))

[학습셋] : 20000 [valset] : 5000 [testset] : 25000 [단어수] : 46159 [class ]: 2


In [0]:
## rnn 모델 구현 

class basic_gru(nn.Module):
  def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    
    super(basic_gru,self).__init__()
    print("building basic gru model ~ ")
    ## n_layers : 은닉 벡터들의 층 
    self.n_layers = n_layers
   
    self.embed = nn.Embedding(n_vocab,embed_dim)
    ## nn.embedding(a,b) a:전체 데이터 셋의 모든 단어를 사전 형태로 나태냈을때 그 수 
    ## b: 임베딩된 단어가 갖는 차원 값 
   
    self.hidden_dim = hidden_dim
    self.dropout = nn.Dropout(dropout_p)
    self.gru = nn.GRU(embed_dim,self.hidden_dim,num_layers= self.n_layers,batch_first=True)
    ## nn.module에 있는 gru 가져다 쓰는것 
   
    self.output = nn.Linear(self.hidden_dim, n_classes)

  def forward(self,x):
   
    x = self.embed(x)
    ##한 문장을 각 단어별로 임베딩 한 것 -> x 
   
    ## 첫번째 은닉벡터 h0를 정으 ㅣ해 x와 함꼐 입력해줘야한다 
    h_0 = self._init_state(batch_size=x.size(0)) ##x의 첫번째 임베딩값을 넣는다 
   
    ##self.gru(x,h_0) => (batch_size, 입력x의 길이, hidden_dim)의 모양을 지닌 3d 텐서로 나옴 
    x,_ = self.gru(x,h_0)
    
    h_t = x[:,-1,:] ## -> 배치 내 모든 시계열 은닉 벡터들의 마지막 토큰들을 내포한 (batch_size,1,hidden_dim)의 텐서 추출  
    ## h_t == 영화 리뷰 배열들을 압축한 은닉 벡터 
   # print("h_t : {}".format(h_t))
    self.dropout(h_t)
    
    logit = self.output(h_t) ## output == linear 
    return logit

  def _init_state(self,batch_size = 1):
    ## parameter함수는 nn.module의 가중치 정보들을 반복자 형태로 반환 
    ## 이 반복자가 생성하는 원소들은 각각 실제 신경망의 가중치 텐서(.data)를 지닌 객체들 

    weight = next(self.parameters()).data ## -> nn.gru 모듈의 첫번째 가중치 텐서를 추출한다 
    ## new() 호출하여 모델으 ㅣ가중치와 같은 모양인 텐서로 바꾼후 zero_()를 호출하여 0으로 초기화 
    return weight.new(self.n_layers,batch_size,self.hidden_dim).zero_()



In [0]:
## train 과 evaluation 

def train(model, optimizer,train_iter):
  model.train()
  for b,batch in enumerate(train_iter):
    ##반복마다 배치데이터를 반환 
    x,y = batch.text.to(DEVICE),batch.label.to(DEVICE)
    y.data.sub_(1) ## label을 0~1로 변환 

    ## 각 배치마다 기울기를 새로 계산 
    optimizer.zero_grad()
#    print("before train model ")
    logit = model(x)
#    print("after model ")
    loss = F.cross_entropy(logit,y) 
    loss.backward() ##기울기 계산하고 ~ 
    optimizer.step() ## 학습하고 ~~ 


## valset과 testset의 성능 측정 
def evaluate(model,val_iter):
  model.eval()
  corrects, total_loss = 0,0
  for batch in val_iter:
    x,y = batch.text.to(DEVICE),batch.label.to(DEVICE)
    y.data.sub_(1)## label 0~1로 바꾸기 

    logit = model(x)
    loss = F.cross_entropy(logit,y,reduction='sum') ## reduction = sum -> 오차의 합을 구함  
    total_loss += loss.item() ## 배치사이즈 만큼 데이터의 오류 다 합친거
    
    corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
  
  size = len(val_iter.dataset)
  avg_loss = total_loss / size
  avg_accuracy = 100.0 * corrects/ size
  
  return avg_loss, avg_accuracy
    

In [73]:
## 학습 전 모델 객체 정의 
## 은닉벡터 차원값 = 256 ,임베딩 토큰 차원값= 128 
model = basic_gru(n_layers=1,hidden_dim=256,n_vocab= vocab_size,embed_dim=128,n_classes=n_classes,dropout_p=0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)


building basic gru model ~ 


In [76]:
## 학습 실행 ㄱㄱ ~ 
best_val_loss = None
checkpoint_dir = "/content/gdrive/My Drive/Colab Notebooks/pytorch/RNN"

os.path.join(checkpoint_dir)

for e in range(1,EPOCHS+1):
  train(model,optimizer,train_iter)
  val_loss, val_accuarcy = evaluate(model,val_iter)

  print("epoch : {} val_loss : {} || val_accuarcy  {}".format(e,val_loss,val_accuarcy))

  ##목표 : 학습 오차가 아닌 검증 오차가 최소화된 모델 을 저장해야한다 
  if not best_val_loss or val_loss < best_val_loss :
    
    torch.save(model.state_dict(),checkpoint_dir+'/txtclassification.pt')
    best_val_loss = val_loss


epoch : 1 val_loss : 0.4356383192062378 || val_accuarcy  86.65999603271484
epoch : 2 val_loss : 0.43324092388153074 || val_accuarcy  87.37999725341797
epoch : 3 val_loss : 0.4510982828855514 || val_accuarcy  86.65999603271484
epoch : 4 val_loss : 0.45810309896469115 || val_accuarcy  87.07999420166016
epoch : 5 val_loss : 0.4826218725681305 || val_accuarcy  87.41999816894531
epoch : 6 val_loss : 0.4619948860168457 || val_accuarcy  87.54000091552734
epoch : 7 val_loss : 0.4785803104400635 || val_accuarcy  87.15999603271484
epoch : 8 val_loss : 0.4840695296764374 || val_accuarcy  86.95999908447266
epoch : 9 val_loss : 0.555582232284546 || val_accuarcy  87.5
epoch : 10 val_loss : 0.5797550442695618 || val_accuarcy  86.25999450683594
epoch : 11 val_loss : 0.5544119689941406 || val_accuarcy  86.5999984741211
epoch : 12 val_loss : 0.6303966052532196 || val_accuarcy  85.86000061035156
epoch : 13 val_loss : 0.6353427826881408 || val_accuarcy  85.86000061035156
epoch : 14 val_loss : 0.5316725960

In [77]:
model.load_state_dict(torch.load(checkpoint_dir + '/txtclassification.pt'))
test_loss , test_acc = evaluate(model,test_iter)
print("test loss : {} test accuaracy : {}".format(test_loss,test_acc))

test loss : 0.4165056896877289 test accuaracy : 86.51200103759766
